In [1]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm

In [2]:
page_types = [
    "https://24.hu/kultura/",
    "https://24.hu/fn/uzleti-tippek/",
    "https://24.hu/fn/gazdasag/",
    "https://24.hu/kulfold/",
    "https://24.hu/kozelet/",
    "https://24.hu/belfold/",
    "https://24.hu/tudomany/",
    "https://24.hu/tech/",
    "https://24.hu/kistotal/",
    "https://24.hu/velemeny/",
    "https://24.hu/szorakozas/",
    "https://24.hu/otthon/",
    "https://24.hu/elet-stilus/",
    "https://24.hu/sport/",
    "https://24.hu/szorakozas/"
]

In [5]:
hu24_dict = {}
for page_type in tqdm(page_types):
    response = requests.get(page_type)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        maxpage = int([page_num.text for page_num in soup.find_all('a', class_='page-numbers')][1].replace(",",""))
    except:
        maxpage = 2
    for page_number in tqdm(range(1, min(4, maxpage))): ### ezt kellene majd átírni maxpage-re
        page_link = page_type + f"page/{page_number}"
        response = requests.get(page_link)
        soup = BeautifulSoup(response.text, 'html.parser')
        articles = soup.select(".m-entryPost__link")
        articles = [article.get("href") for article in articles]
        
        for article in articles:
            response = requests.get(article)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                date = soup.find('div', class_='m-author__wrapCatDateTitulus').text.replace("\t", "").replace("\n","")
                paragraphs = soup.select(".post-body p")

                related_urls = []
                for paragraph in paragraphs:
                    if "MTI" in paragraph.text or "mti" in paragraph.text or "Mti" in paragraph.text:
                        related_urls.append("MTI")
                    if paragraph.find_all("a") is not None:
                        for link in paragraph.find_all("a"):
                            if "http" in link.get('href'):
                                related_urls.append(link.get('href'))
                            else:
                                related_urls.append("https:" + link.get('href'))
                hu24_dict[article] = {
                        "related_links" : related_urls,
                        "publish_date" : date
                        #"last_modification_date" : dik[0]["_mod_ts"]
                    }
            else:
                print(status_code)